In [1]:
import torch
import numpy as np
import torch.nn as nn

In [2]:
import importnb
with __import__('importnb').Notebook(): 
    from tools import ScaledDotProductAttention
    from tools import MultiHeadAttention
    from tools import AddPositionalEncoding
    from tools import TransformerFNN

## SelfAttentionの実行

### 適当なデータの作成
---
新しくデータkを作る。kは***(B,T,d_model)***のshapeを持つテンソルである。


### SelfAttentionにする
---
q,k,vを同じテンソルにすることでSelfAttentionにする。


### forwardで計算を行う。
---
呼び出したspa.forward()によって計算を行う。
この時spaの初期化で与えるd_kには本来d_modelをhead数で割った値が入る(デフォルトだと512/8で64)

In [14]:
###SelfAttentionの実行
num_head = 4
batch_size = 2
seq_len = 10
d_model = 64
max_len = 512
k = torch.randn(batch_size,seq_len,d_model)

In [15]:
###SelfAttentionにするためにqとvとkを一緒にする
pe = AddPositionalEncoding(d_model,max_len)
k = pe(k)
q = k
v = k
spa = ScaledDotProductAttention(16)
attention_weight = spa.forward(q,k,v)

In [16]:
attention_weight.shape

torch.Size([2, 10, 64])

## MultiHeadAttentionの実行
ランダムなテンソル、スタンダードなマスクを使う。

In [17]:
def create_incremental_mask(seq_len):
    """
    seq_len x seq_len のサイズのマスクを生成する。
    0列目は全てFalse、以降の列では上から順にTrueの数を増やしていく。
    """
    # seq_len x seq_len の行列を生成し、初期値は全てFalseに設定
    mask = torch.full((seq_len, seq_len), False)

    # 各列に対して、上から順にTrueをセットする
    for i in range(seq_len):
        mask[:i, i] = True

    return mask
mask = create_incremental_mask(seq_len).repeat(batch_size,1,1)
mask.shape

torch.Size([2, 10, 10])

In [18]:
mha = MultiHeadAttention(num_head,d_model)
output = mha(q,k,v,mask=mask)

In [19]:
output.shape

torch.Size([2, 10, 64])

In [20]:
output

tensor([[[ 0.1116,  0.4256,  0.2973,  ...,  0.0891, -0.3531,  0.1125],
         [-0.1245,  0.2894,  0.1675,  ...,  0.1052, -0.1104,  0.1493],
         [-0.0582,  0.2513,  0.1443,  ...,  0.2361, -0.0443,  0.2246],
         ...,
         [-0.1632,  0.1023,  0.0091,  ...,  0.1978,  0.0255,  0.0438],
         [-0.1635,  0.0764,  0.0023,  ...,  0.2245,  0.0366,  0.0202],
         [-0.1214,  0.1232,  0.0191,  ...,  0.2253,  0.0042, -0.0100]],

        [[-0.0242,  0.4335, -0.1325,  ...,  0.2754, -0.3820, -0.3835],
         [-0.0508,  0.3018, -0.0374,  ...,  0.2524, -0.1548, -0.2066],
         [-0.0208,  0.3573,  0.0093,  ...,  0.3498, -0.1471, -0.2391],
         ...,
         [-0.0161,  0.2141,  0.0298,  ...,  0.1064, -0.0140, -0.2941],
         [-0.0066,  0.2311,  0.0255,  ...,  0.1294, -0.0374, -0.2716],
         [-0.0407,  0.2099,  0.0349,  ...,  0.1324, -0.0149, -0.2273]]],
       grad_fn=<ViewBackward0>)